In [0]:
%sql
-- 1 Cria o catálogo principal (onde toda a plataforma de dados será organizada)
CREATE CATALOG IF NOT EXISTS lakehouse
COMMENT "Catálogo principal do projeto, com camadas de dados governadas via Unity Catalog";

-- 2 Cria as quatro camadas da arquitetura de dados
CREATE SCHEMA IF NOT EXISTS lakehouse.raw
COMMENT "Camada RAW - dados brutos, como chegam das fontes (SQL, API, planilhas)";
CREATE VOLUME IF NOT EXISTS lakehouse.raw.coinbase;
CREATE VOLUME IF NOT EXISTS lakehouse.raw.yfinance;

CREATE SCHEMA IF NOT EXISTS lakehouse.bronze
COMMENT "Camada BRONZE - dados padronizados, com metadados e controle de ingestão";

CREATE SCHEMA IF NOT EXISTS lakehouse.silver
COMMENT "Camada SILVER - dados tratados, com regras de negócio aplicadas";

CREATE SCHEMA IF NOT EXISTS lakehouse.gold
COMMENT "Camada GOLD - dados analíticos e métricas finais para BI e IA";

-- 3 (opcional) Verifica se tudo foi criado corretamente
SHOW SCHEMAS IN lakehouse;

In [0]:
%sql
-- 2) Permissões (um GRANT por principal)

-- Catálogo
GRANT USE CATALOG ON CATALOG lakehouse TO data_engineers;
GRANT USE CATALOG ON CATALOG lakehouse TO data_analysts;
GRANT USE CATALOG ON CATALOG lakehouse TO data_scientists;
GRANT USE CATALOG ON CATALOG lakehouse TO business_users;

-- RAW (restrito a engenheiros)
GRANT USE SCHEMA ON SCHEMA lakehouse.raw TO data_engineers;
GRANT SELECT ON SCHEMA lakehouse.raw TO data_engineers;

-- BRONZE (eng: criar/modificar; analistas: leitura)
GRANT USE SCHEMA ON SCHEMA lakehouse.bronze TO data_engineers;
GRANT USE SCHEMA ON SCHEMA lakehouse.bronze TO data_analysts;
-- GRANT SELECT ON ALL TABLES IN SCHEMA lakehouse.bronze TO data_analysts;

-- SILVER (leitura p/ times técnicos)
GRANT USE SCHEMA ON SCHEMA lakehouse.silver TO data_engineers;
GRANT USE SCHEMA ON SCHEMA lakehouse.silver TO data_analysts;
GRANT USE SCHEMA ON SCHEMA lakehouse.silver TO data_scientists;
-- GRANT SELECT ON ALL TABLES IN SCHEMA lakehouse.silver TO data_engineers;
-- GRANT SELECT ON ALL TABLES IN SCHEMA lakehouse.silver TO data_analysts;
-- GRANT SELECT ON ALL TABLES IN SCHEMA lakehouse.silver TO data_scientists;

-- GOLD (leitura ampla, incluindo negócio)
GRANT USE SCHEMA ON SCHEMA lakehouse.gold TO data_engineers;
GRANT USE SCHEMA ON SCHEMA lakehouse.gold TO data_analysts;
GRANT USE SCHEMA ON SCHEMA lakehouse.gold TO data_scientists;
GRANT USE SCHEMA ON SCHEMA lakehouse.gold TO business_users;
-- GRANT SELECT ON ALL TABLES IN SCHEMA lakehouse.gold TO data_engineers;
-- GRANT SELECT ON ALL TABLES IN SCHEMA lakehouse.gold TO data_analysts;
-- GRANT SELECT ON ALL TABLES IN SCHEMA lakehouse.gold TO data_scientists;
-- GRANT SELECT ON ALL TABLES IN SCHEMA lakehouse.gold TO business_users;

In [0]:
%sql

-- Permissões no CATÁLOGO
GRANT USE CATALOG ON CATALOG lakehouse TO `account users`;
GRANT USE SCHEMA ON CATALOG lakehouse TO `account users`;
GRANT CREATE SCHEMA ON CATALOG lakehouse TO `account users`;

-- Permissões no SCHEMA
GRANT USE SCHEMA ON SCHEMA lakehouse.raw TO `account users`;
GRANT CREATE TABLE ON SCHEMA lakehouse.raw TO `account users`;
GRANT SELECT ON SCHEMA lakehouse.raw TO `account users`;
GRANT MODIFY ON SCHEMA lakehouse.raw TO `account users`;

-- Permissões no VOLUME (importante pro seu caso!)
GRANT READ VOLUME ON VOLUME lakehouse.raw.coinbase TO `account users`;
GRANT WRITE VOLUME ON VOLUME lakehouse.raw.coinbase TO `account users`;

In [0]:
%sql
-- Grant USE CATALOG privilege
GRANT USE CATALOG ON CATALOG lakehouse TO `data_engineers`;

-- Grant USE SCHEMA privilege
GRANT USE SCHEMA ON SCHEMA lakehouse.raw TO `data_engineers`;

-- (Optional) Grant CREATE TABLE privilege if the user/group needs to create tables
GRANT CREATE TABLE ON SCHEMA lakehouse.raw TO `data_engineers`;